# Name: Anjali Nagpal
# Id: 8898795

 Load the dataset into a pandas DataFrame

In [8]:

import pandas as pd

# Load the dataset into a pandas DataFrame
data = pd.read_csv(r'C:\Users\Arin\Downloads\archive (2)\heart.csv') 


Load the dataset into a pandas DataFrame

In [9]:

from scipy.stats import zscore
# Select only the numeric columns for Z score calculation
numeric_columns = data.select_dtypes(include=[int, float]).columns
data_numeric = data[numeric_columns]

# Calculate Z scores for each column
z_scores = zscore(data_numeric)
# Filter out rows where the absolute Z score is greater than 3
data_no_outliers = data[(z_scores < 3).all(axis=1)]


Convert text columns to numbers using label encoding and one hot encoding

In [10]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoders = {}

# Label encoding for categorical columns
for column in ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']:
    le = LabelEncoder()
    data_no_outliers[column] = le.fit_transform(data_no_outliers[column])
    label_encoders[column] = le

# One hot encoding for selected categorical columns
data_encoded = pd.get_dummies(data_no_outliers, columns=['ChestPainType', 'RestingECG', 'ST_Slope'])


C:\Users\Arin\AppData\Local\Temp\ipykernel_22576\700073741.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_outliers[column] = le.fit_transform(data_no_outliers[column])
C:\Users\Arin\AppData\Local\Temp\ipykernel_22576\700073741.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_outliers[column] = le.fit_transform(data_no_outliers[column])
C:\Users\Arin\AppData\Local\Temp\ipykernel_22576\700073741.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

 Apply scaling

In [11]:


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_encoded.drop('HeartDisease', axis=1))
scaled_df = pd.DataFrame(scaled_data, columns=data_encoded.columns[:-1])


Build SVM classification models (standalone and Bagging)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

X = scaled_df.values
y = data_encoded['HeartDisease'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build standalone SVM model
svm_standalone = SVC(kernel='linear')
svm_standalone.fit(X_train, y_train)

# Build SVM model with Bagging
svm_bagging = BaggingClassifier(base_estimator=SVC(kernel='linear'), n_estimators=10)
svm_bagging.fit(X_train, y_train)


C:\Users\Arin\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=SVC(kernel='linear'))

Build Decision Tree classification models (standalone and Bagging)

In [13]:
from sklearn.tree import DecisionTreeClassifier

# Build standalone Decision Tree model
dt_standalone = DecisionTreeClassifier(random_state=42)
dt_standalone.fit(X_train, y_train)

# Build Decision Tree model with Bagging
dt_bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42), n_estimators=10)
dt_bagging.fit(X_train, y_train)


C:\Users\Arin\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42))

Compare SVM and Decision Tree classifier performance

In [14]:
from sklearn.metrics import accuracy_score

# Predictions for SVM models
svm_standalone_pred = svm_standalone.predict(X_test)
svm_bagging_pred = svm_bagging.predict(X_test)

# Predictions for Decision Tree models
dt_standalone_pred = dt_standalone.predict(X_test)
dt_bagging_pred = dt_bagging.predict(X_test)

# Calculate accuracies
svm_standalone_accuracy = accuracy_score(y_test, svm_standalone_pred)
svm_bagging_accuracy = accuracy_score(y_test, svm_bagging_pred)

dt_standalone_accuracy = accuracy_score(y_test, dt_standalone_pred)
dt_bagging_accuracy = accuracy_score(y_test, dt_bagging_pred)

print("SVM Standalone Accuracy:", svm_standalone_accuracy)
print("SVM Bagging Accuracy:", svm_bagging_accuracy)

print("Decision Tree Standalone Accuracy:", dt_standalone_accuracy)
print("Decision Tree Bagging Accuracy:", dt_bagging_accuracy)


SVM Standalone Accuracy: 0.861878453038674
SVM Bagging Accuracy: 0.850828729281768
Decision Tree Standalone Accuracy: 0.8397790055248618
Decision Tree Bagging Accuracy: 0.8397790055248618
